<a href="https://colab.research.google.com/github/DrSubbiah/python/blob/main/111CallGeneratingCondiotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pathlib
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from datetime import datetime, date, timedelta

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set path (drive)

path = r'/content/drive/MyDrive/XTS Data_Message Files/221019'

#Fix the file pattern
filenames = glob.glob(path + "/*.csv")

#Read all files
dfs = [pd.read_csv(filename).assign(Source=os.path.basename(filename)) for filename in filenames]

#Concatenate all data into one DataFrame

df = pd.concat(dfs, ignore_index=False)

In [ ]:
df

,SYMBOL,Prev Signals,Signal,Signal.Condition,Trade Condition,Depth,Open,High,Low,Close,...,Source,Unnamed: 0,buy_c1,buy_c2,new_buy_c2,sell_c1,sell_c2,new_sell_c2,depth_abs,Source1
0,HAPPSTMNDS,BUY,Sell,False,False,-991.4,988.8,992.0,991.4,991.4,...,10_28_09_message.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HDFCBANK,BUY,Sell,False,False,-1467.8,1443.8,1468.2,1466.4,1467.8,...,10_28_09_message.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KOTAKBANK,BUY,Sell,False,False,-1868.2,1859.7,1869.0,1868.0,1868.2,...,10_28_09_message.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ADANIPORTS,SELL,Sell,False,False,-819.8,810.0,820.5,818.7,819.8,...,10_28_09_message.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AUROPHARMA,BUY,Sell,False,False,-520.9,517.7,520.9,520.1,520.9,...,10_28_09_message.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,INDIACEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,conditions1.csv,129.0,pass,pass,fail,fail,pass,pass,0.5,150913.0
130,INFY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,conditions1.csv,130.0,pass,pass,fail,fail,pass,pass,0.9,150913.0
131,L&TFH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,conditions1.csv,131.0,fail,pass,pass,pass,pass,fail,0.1,150913.0
132,RADICO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,conditions1.csv,132.0,pass,pass,fail,fail,pass,pass,0.4,150913.0


In [ ]:
df = final_df

Checking and filtering out rows which dont satisfy the condition

In [ ]:
df = df.loc[(df['OC_CONDITION'] == 'CLOSE') & (df['Sleep_Condition'] == True) & (df['Cycle'] % 28 == 0)]

MADE CHANGES TO CONDITION

FROM '+' TO '-' IN df['scon1]

In [ ]:
B_S_DIST = 0.25
df['bcon1'] = df['PP'] + (B_S_DIST*(df['PR'] - df['PP']))
df['scon1'] = df['PP'] - (B_S_DIST*(df['PP'] - df['PS']))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
L1 = 150
L2 = 1000
L3 = 5000

np.select and df.assign was not working, tried fixing it to no real success

In [ ]:
# def func(x):
#   x['Close'] >= x['PP']
#   return x

# buyyy = df.groupby('Close',as_index=False)['PP']
# buyyy
#df = df.assign(buy_c1 = ('pass' if all(df.Close) >= all(df.PP) else 'fail'))
#df = df.assign(r= np.select([df.Close <= L1, [(L1 < df.Close) & (df.Close <= L2)], [(L2 < df.Close) & (df.Close <= L3)], L3 < df.Close], ['0.0149', "0.0108", '0.005', '0.0033']))
df = (df.assign(r= lambda x: pd.cut(x['Close'], 
                                              bins=[0, 150, 1000, 5000,100000],
                                              labels=[0.0149, 0.0108, 0.005, 0.0033])))
df['buy_c1'] = np.where(df['Close'] >= df['PP'], "pass", "fail")
df['buy_c2'] = np.where(df['Close'] <= df['bcon1'], "pass", "fail")
df['sell_c1'] = np.where(df['Close'] <= df['PP'], "pass", "fail")
df['sell_c2'] = np.where(df['Close'] >= df['scon1'], "pass", "fail")


In [ ]:
df.dtypes
df_copy1 = df.copy()
df['r'] = df.r.astype(str)
df['r'] = df.r.astype(float)
df['PR_1'] = df['Close'] + (1+df['r'])
df['PS_1'] = df['Close'] + (1-df['r'])
df['new_bcon1'] = df['PP'] + (B_S_DIST*(df['PR_1'] - df['PP']))
df['new_scon1'] = df['PP'] + (B_S_DIST*(df['PP'] - df['PS_1']))
df['new_buy_c2'] = np.where(df['Close'] <= df['new_bcon1'], "pass", "fail")
df['new_sell_c2'] = np.where(df['Close'] >= df['new_scon1'], "pass", "fail")

All the commented out code below was me doing it manually, then figured out how to do it np.where

In [ ]:
# buy_pp = df.loc[(df['PP'] <= df['Close'])]
# nbuy_pp = df.loc[(df['PP'] > df['Close'])]
# a = nbuy_pp['SYMBOL'].index
# buy_pp1 = buy_pp['SYMBOL'].index
# print('failed pp:\t',a,'\n',
#       'pp succesful:\t',buy_pp1)
# buy_con = df.loc[(df['Close'] <= df['con1'])]
# nbuy_con = df.loc[(df['Close'] > df['con1'])]
# buy_con1 = buy_con['SYMBOL'].index
# b = nbuy_con['SYMBOL'].index
# print('\n\n\nfailed con1:\t',b,
#       '\ncon1 successful:\t',buy_con1)
# ppp = df.loc[(df['PP'] < df['Close']) & (df['Close'] <= df['con1'])]
# c = ppp['SYMBOL'].index
# ccc = df.loc[(df['PP'] >= df['Close']) & (df['Close'] > df['con1'])]
# d = ccc['SYMBOL'].index
# print('\n\n\nfailed pp but con1 =T:\t',c,
#       '\ncon1=F but pp successful:\t',d)

In [ ]:
# buy = df.loc[(df['Close'] >= df['PP']) & (df['Close'] <= df['con1'])]
# buy1 = df.loc[~(df['Close'] >= df['PP']) & (df['Close'] <= df['con1'])]
# print(buy['SYMBOL'].index,'\n\t',buy1['SYMBOL'].index)

In [ ]:
# sell = df.loc[(df['Close'] >= df['con1']) & (df['Close'] <= df['PP'])]
# sell
# sell_con = df.loc[(df['Close'] >= df['con1'])]
# nsell_con = df.loc[~(df['Close'] >= df['con1'])]
# sell_pp = df.loc[(df['Close'] <= df['PP'])]
# nsell_pp = df.loc[~(df['Close'] <= df['PP'])]
# print(sell_pp['SYMBOL'].index,'\n\t',nsell_pp['SYMBOL'].index)


In [ ]:
# df['buy con1'] = ''
# df['buy con1'][0, 5, 6, 8, 9, 16, 18] = 'fail'
# df['buy con1'][1, 2, 3, 4, 7, 10, 11, 12, 13, 14, 15, 17, 19] = 'pass'

# df['buy con2'] = ''
# df['buy con2'][2, 4, 13, 14, 15, 17, 19] = 'fail'
# df['buy con2'][0, 1, 3, 5, 6, 7, 8, 9, 10, 11, 12, 16, 18] = 'pass'
# df['sell con1'] = ''
# df['sell con2'] = ''
# df['sell con1'][1, 2, 3, 4, 7, 10, 11, 12, 13, 14, 15, 17, 19] = 'fail'
# df['sell con1'][0, 5, 6, 8, 9, 16, 18] = 'pass'
# df['sell con2'][0, 1, 3, 5, 6, 7, 8, 9, 10, 11, 16, 18] = 'fail'
# df['sell con2'][2, 4, 12, 13, 14, 15, 17, 19] = 'pass'
# df
# df11 = df[['SYMBOL', 'buy con1','buy con2', 'sell con1', 'sell con2']]

Cleaning out the time column

In [ ]:
df['depth_abs'] = abs(df['Depth'])
df['Source'] = df['Source'].str.rstrip('_message.csv')
df['Source'] = df['Source'].str.strip('_')

Taking only the columns we need for the final df

In [ ]:
BUYY = df[['SYMBOL', 'buy_c1', 'buy_c2', 'new_buy_c2', 'sell_c1', 'sell_c2', 'new_sell_c2', 'depth_abs', 'Source']]
dff = BUYY

In [ ]:
dff

,SYMBOL,buy_c1,buy_c2,new_buy_c2,sell_c1,sell_c2,new_sell_c2,depth_abs,Source
0,HAPPSTMNDS,pass,fail,fail,fail,pass,pass,991.4,10_28_09
1,HDFCBANK,pass,fail,fail,fail,pass,pass,1467.8,10_28_09
2,KOTAKBANK,pass,fail,fail,fail,pass,pass,1868.2,10_28_09
3,ADANIPORTS,pass,fail,fail,fail,pass,pass,819.8,10_28_09
4,AUROPHARMA,pass,fail,fail,fail,pass,pass,520.9,10_28_09
...,...,...,...,...,...,...,...,...,...
15,INDIACEM,pass,pass,fail,fail,pass,pass,0.5,15_09_13
16,INFY,pass,pass,fail,fail,pass,pass,0.9,15_09_13
17,L&TFH,fail,pass,pass,pass,pass,fail,0.1,15_09_13
18,RADICO,pass,pass,fail,fail,pass,pass,0.4,15_09_13


Sorting the df by meaningful values

In [ ]:
#print(BUYY[['new_buy_c2']].to_string(index=False))
dff = dff.sort_values(by=['Source', 'SYMBOL'], ascending=[True, True])
dff = dff.replace({'Source': {'_': ':'}}, regex=True)
# dff['Source'] = pd.to_datetime(dff.Source).dt.time
dff = dff.reset_index(drop = True)

In [ ]:
# dff['Source'] = dff['Source'].astype('str')
# dff = dff.replace({'Source': {':': ''}}, regex=True)
# dff['Source'] = dff['Source'].astype('int')
dff.drop(dff.columns[[1,2,3]], axis = 1)
dff.drop(dff.columns[[4,5,6]], axis = 1)

,SYMBOL,buy_c1,buy_c2,new_buy_c2,depth_abs,Source
0,ADANIPORTS,pass,fail,fail,819.8,10:28:09
1,AUROPHARMA,pass,fail,fail,520.9,10:28:09
2,AXISBANK,pass,fail,fail,831.0,10:28:09
3,BERGEPAINT,pass,fail,fail,627.0,10:28:09
4,HAPPSTMNDS,pass,fail,fail,991.4,10:28:09
...,...,...,...,...,...,...
129,INDIACEM,pass,pass,fail,0.5,15:09:13
130,INFY,pass,pass,fail,0.9,15:09:13
131,L&TFH,fail,pass,pass,0.1,15:09:13
132,RADICO,pass,pass,fail,0.4,15:09:13


Was trying to compare the timeData to a raw time values. Still do not know how to. Had to workaround by making the time column an int value and then compared it to an int value instead. This will not work for dates.

In [ ]:
# dff['next'] = dff.groupby(['SYMBOL'])['Source'].shift(-1)

In [ ]:
# dff['Time_Diff'] = pd.to_datetime(dff.next) - pd.to_datetime(dff.Source)
# dff

In [ ]:
# dff.dtypes['next']

In [ ]:
dff['Source1'] = dff['Source'].astype('str')
dff = dff.replace({'Source1': {':': ''}}, regex=True)
dff['Source1'] = dff['Source1'].astype('int')

0      102809
20     105607
35     112407
48     115209
61     122007
74     124913
87     131711
97     134515
107    141309
116    144107
125    150913
Name: Source1, dtype: int64

In [ ]:
dff.Source.drop_duplicates()

0      10:28:09
20     10:56:07
35     11:24:07
48     11:52:09
61     12:20:07
74     12:49:13
87     13:17:11
97     13:45:15
107    14:13:09
116    14:41:07
125    15:09:13
Name: Source, dtype: object

In [ ]:
cond1 = dff.Source1 < 134515
dff.loc[cond1]

,SYMBOL,buy_c1,buy_c2,new_buy_c2,sell_c1,sell_c2,new_sell_c2,depth_abs,Source,Source1
0,ADANIPORTS,pass,fail,fail,fail,pass,pass,819.8,10:28:09,102809
1,AUROPHARMA,pass,fail,fail,fail,pass,pass,520.9,10:28:09,102809
2,AXISBANK,pass,fail,fail,fail,pass,pass,831.0,10:28:09,102809
3,BERGEPAINT,pass,fail,fail,fail,pass,pass,627.0,10:28:09,102809
4,HAPPSTMNDS,pass,fail,fail,fail,pass,pass,991.4,10:28:09,102809
...,...,...,...,...,...,...,...,...,...,...
92,INDIACEM,pass,pass,pass,fail,pass,pass,0.2,13:17:11,131711
93,INFY,fail,pass,pass,pass,pass,fail,0.4,13:17:11,131711
94,L&TFH,pass,pass,pass,pass,pass,pass,0.0,13:17:11,131711
95,RADICO,fail,pass,pass,pass,pass,fail,0.6,13:17:11,131711


Writing csv

In [ ]:
dff.to_csv(path + "/conditions1.csv")